# Tensorflow 前端

参考: [](https://xinetzone.github.io/tvm/docs/arch/frontend/tensorflow.html)

```{note}
请将 `tensorflow` 的 GPU 内存使用限制在必要的范围内，而不是使用所有可用的内存。您可以参考 [limiting_gpu_memory_growth](https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth) 了解如何进行操作。
```

In [1]:
import tvm
from tvm import te
from tvm import relay
import numpy as np
import tensorflow as tf
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("tensorflow will use experimental.set_memory_growth(True)")
    except RuntimeError as e:
        print("experimental.set_memory_growth option is not available: {}".format(e))

try:
    tf_compat_v1 = tf.compat.v1
except (ImportError, AttributeError):
    tf_compat_v1 = tf

# Tensorflow 实用函数
import tvm.relay.testing.tf as tf_testing

2023-06-08 23:08:41.781065: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 23:08:41.831159: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 23:08:42.631039: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-08 23:08:43.880828: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## 准备阶段

In [2]:
# 模型相关文件的基础位置。
repo_base = "https://github.com/dmlc/web-data/raw/main/tensorflow/models/InceptionV1"
# 测试图片
img_name = "elephant-299.jpg"
image_url = f"{repo_base}/{img_name}"
# 模型
model_name = "classify_image_graph_def-with_shapes.pb"
model_url = f"{repo_base}/{model_name}"
# 图像标签映射
map_proto = "imagenet_2012_challenge_label_map_proto.pbtxt"
map_proto_url = f"{repo_base}/{map_proto}"
# 标签的人类可读文本。
label_map = "imagenet_synset_to_human_label_map.txt"
label_map_url = f"{repo_base}/{label_map}"

下载如下文件：

In [3]:
from tvm.contrib.download import download_testdata

img_path = download_testdata(image_url, img_name, module="data")
model_path = download_testdata(model_url, model_name, module=["tf", "InceptionV1"])
map_proto_path = download_testdata(map_proto_url, map_proto, module="data")
label_path = download_testdata(label_map_url, label_map, module="data")

## 在 tensorflow 上推理

在 TensorFlow 上运行相应的模型。

In [4]:
def create_graph(model_path):
    """从保存的 GraphDef 文件创建图并返回 saver。"""
    # 从保存的 graph_def.pb 文件创建图。
    with tf_compat_v1.gfile.GFile(model_path, "rb") as f:
        graph_def = tf_compat_v1.GraphDef()
        graph_def.ParseFromString(f.read())
        graph = tf.import_graph_def(graph_def, name="")
        # 调用实用程序将图定义导入默认图中。
        graph_def = tf_testing.ProcessGraphDefParam(graph_def)
        # 在图中添加形状。
        with tf_compat_v1.Session() as sess:
            graph_def = tf_testing.AddShapesToGraphDef(sess, "softmax")
    return graph_def

def run_inference_on_image(model_path, image_path):
    """对图像进行推理。

    Parameters
    ----------
    image: String
        Image file name.

    Returns
    -------
        Nothing
    """
    if not tf_compat_v1.gfile.Exists(image_path):
        tf.logging.fatal("File does not exist %s", image_path)
    with tf_compat_v1.gfile.GFile(image_path, "rb") as img_f:
        image_data = img_f.read()

    # 从已保存的 GraphDef 创建图。
    create_graph(model_path)

    with tf_compat_v1.Session() as sess:
        softmax_tensor = sess.graph.get_tensor_by_name("softmax:0")
        predictions = sess.run(softmax_tensor, {"DecodeJpeg/contents:0": image_data})
        predictions = np.squeeze(predictions)
        # 创建节点ID --> 英文字符串查找。
        node_lookup = tf_testing.NodeLookup(
            label_lookup_path=map_proto_path, uid_lookup_path=label_path
        )
        # 打印 tensorflow 的 top5
        top_k = predictions.argsort()[-5:][::-1]
        print("===== TENSORFLOW 结果 =======")
        for node_id in top_k:
            human_string = node_lookup.id_to_string(node_id)
            score = predictions[node_id]
            print(f"{human_string} (score = {score:.5f})")


run_inference_on_image(model_path, img_path)

2023-06-08 23:08:44.759192: W tensorflow/core/framework/op_def_util.cc:369] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().


Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


2023-06-08 23:08:45.348685: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled


===== TENSORFLOW 结果 =======
African elephant, Loxodonta africana (score = 0.58394)
tusker (score = 0.33909)
Indian elephant, Elephas maximus (score = 0.03186)
banana (score = 0.00022)
desk (score = 0.00019)


## Relay 推理

将 TensorFlow graph 定义导入到 Relay 前端。

结果：

- sym: Relay 表达式，表示给定的 tensorflow protobuf。
- params: 从 tensorflow params（张量 protobuf）转换而来的参数。

目标设备设置:

In [5]:
# 使用这些注释设置来构建 cuda
# target = tvm.target.Target("cuda", host="llvm")
# layout = "NCHW"
# dev = tvm.cuda(0)
target = tvm.target.Target("llvm", host="llvm")
layout = None
dev = tvm.cpu(0)

In [6]:
shape = 299, 299, 3
shape_dict = {"DecodeJpeg/contents": shape}
dtype_dict = {"DecodeJpeg/contents": "uint8"}

graph_def = create_graph(model_path)
mod, params = relay.frontend.from_tensorflow(graph_def, layout=layout, shape=shape_dict)
print("TensorFlow 的 protobuf 已导入到 Relay 前端。")

/media/pc/data/lxw/ai/tvm/xinetzone/tvm-book/__pypackages__/3.10/lib/tvm/relay/frontend/tensorflow.py:537: UserWarning: Ignore the passed shape. Shape in graphdef will be used for operator DecodeJpeg/contents.
  warnings.warn(
/media/pc/data/lxw/ai/tvm/xinetzone/tvm-book/__pypackages__/3.10/lib/tvm/relay/frontend/tensorflow_ops.py:1036: UserWarning: DecodeJpeg: It's a pass through, please handle preprocessing before input
  warnings.warn("DecodeJpeg: It's a pass through, please handle preprocessing before input")


TensorFlow 的 protobuf 已导入到 Relay 前端。


### Relay 构建

使用给定的输入规格将图编译为 LLVM 目标。 

结果：
  
- `graph`：编译后的最终计算图。
- `params`：编译后的最终参数。
- `lib`：可以在具有 TVM 运行时的目标上部署的目标库。


In [7]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC layout is not optimized for x86 with autotvm.
conv2d NHWC la

### 在 TVM 上执行 portable graph

现在我们可以尝试在目标设备上部署编译好的模型。

In [8]:
from tvm.contrib import graph_executor
from PIL import Image

image = Image.open(img_path).resize((299, 299))
x = np.array(image)
dtype = "uint8"
m = graph_executor.GraphModule(lib["default"](dev))
# set inputs
m.set_input("DecodeJpeg/contents", tvm.nd.array(x.astype(dtype)))
# execute
m.run()
# get outputs
tvm_output = m.get_output(0)

### TVM 处理输出

将 InceptionV1 模型的输出处理成可读的文本形式。


In [9]:
predictions = tvm_output.numpy()
predictions = np.squeeze(predictions)

# Creates node ID --> English string lookup.
node_lookup = tf_testing.NodeLookup(label_lookup_path=map_proto_path, uid_lookup_path=label_path)
# Print top 5 predictions from TVM output.
top_k = predictions.argsort()[-5:][::-1]
for node_id in top_k:
    human_string = node_lookup.id_to_string(node_id)
    score = predictions[node_id]
    print(f"{human_string} (score = {score:.5f})")

African elephant, Loxodonta africana (score = 0.58335)
tusker (score = 0.33901)
Indian elephant, Elephas maximus (score = 0.02391)
banana (score = 0.00025)
vault (score = 0.00021)


## 布局变换

In [10]:
print(mod["main"])

fn (%DecodeJpeg/contents: Tensor[(299, 299, 3), uint8] /* span=DecodeJpeg/contents:0:0 */, %conv/conv2d_params: Tensor[(3, 3, 3, 32), float32] /* span=conv/conv2d_params:0:0 */, %conv/batchnorm/gamma: Tensor[(32), float32] /* span=conv/batchnorm/gamma:0:0 */, %conv/batchnorm/beta: Tensor[(32), float32] /* span=conv/batchnorm/beta:0:0 */, %conv/batchnorm/moving_mean: Tensor[(32), float32] /* span=conv/batchnorm/moving_mean:0:0 */, %conv/batchnorm/moving_variance: Tensor[(32), float32] /* span=conv/batchnorm/moving_variance:0:0 */, %conv_1/conv2d_params: Tensor[(3, 3, 32, 32), float32] /* span=conv_1/conv2d_params:0:0 */, %conv_1/batchnorm/gamma: Tensor[(32), float32] /* span=conv_1/batchnorm/gamma:0:0 */, %conv_1/batchnorm/beta: Tensor[(32), float32] /* span=conv_1/batchnorm/beta:0:0 */, %conv_1/batchnorm/moving_mean: Tensor[(32), float32] /* span=conv_1/batchnorm/moving_mean:0:0 */, %conv_1/batchnorm/moving_variance: Tensor[(32), float32] /* span=conv_1/batchnorm/moving_variance:0:0 */

In [11]:
# We assume our model's heavily-layout sensitive operators only consist of nn.conv2d
desired_layouts = {
    'image.resize2d': ['NCHW'],
    'nn.conv2d': ['NCHW', 'default'],
    'nn.max_pool2d': ['NCHW', 'default'],
    'nn.avg_pool2d': ['NCHW', 'default'],
}

# Convert the layout to NCHW
# RemoveUnunsedFunctions is used to clean up the graph.
seq = tvm.transform.Sequential([relay.transform.RemoveUnusedFunctions(),
                                relay.transform.ConvertLayout(desired_layouts)])
with tvm.transform.PassContext(opt_level=3):
    mod = seq(mod)

In [12]:
print(mod["main"])

fn (%DecodeJpeg/contents: Tensor[(299, 299, 3), uint8] /* ty=Tensor[(299, 299, 3), uint8] span=DecodeJpeg/contents:0:0 */, %conv/conv2d_params: Tensor[(3, 3, 3, 32), float32] /* ty=Tensor[(3, 3, 3, 32), float32] span=conv/conv2d_params:0:0 */, %conv/batchnorm/gamma: Tensor[(32), float32] /* ty=Tensor[(32), float32] span=conv/batchnorm/gamma:0:0 */, %conv/batchnorm/beta: Tensor[(32), float32] /* ty=Tensor[(32), float32] span=conv/batchnorm/beta:0:0 */, %conv/batchnorm/moving_mean: Tensor[(32), float32] /* ty=Tensor[(32), float32] span=conv/batchnorm/moving_mean:0:0 */, %conv/batchnorm/moving_variance: Tensor[(32), float32] /* ty=Tensor[(32), float32] span=conv/batchnorm/moving_variance:0:0 */, %conv_1/conv2d_params: Tensor[(3, 3, 32, 32), float32] /* ty=Tensor[(3, 3, 32, 32), float32] span=conv_1/conv2d_params:0:0 */, %conv_1/batchnorm/gamma: Tensor[(32), float32] /* ty=Tensor[(32), float32] span=conv_1/batchnorm/gamma:0:0 */, %conv_1/batchnorm/beta: Tensor[(32), float32] /* ty=Tensor[(

In [13]:
# Call relay compilation
with relay.build_config(opt_level=3):
     lib = relay.build(mod, target, params=params)

In [14]:
m = graph_executor.GraphModule(lib["default"](dev))
# set inputs
m.set_input("DecodeJpeg/contents", tvm.nd.array(x.astype(dtype)))
# execute
m.run()
# get outputs
tvm_output = m.get_output(0)

In [15]:
predictions = tvm_output.numpy()
predictions = np.squeeze(predictions)

# Creates node ID --> English string lookup.
node_lookup = tf_testing.NodeLookup(label_lookup_path=map_proto_path, uid_lookup_path=label_path)
# Print top 5 predictions from TVM output.
top_k = predictions.argsort()[-5:][::-1]
for node_id in top_k:
    human_string = node_lookup.id_to_string(node_id)
    score = predictions[node_id]
    print(f"{human_string} (score = {score:.5f})")

African elephant, Loxodonta africana (score = 0.58335)
tusker (score = 0.33901)
Indian elephant, Elephas maximus (score = 0.02391)
banana (score = 0.00025)
vault (score = 0.00021)


In [18]:
relay._ffi_api.Any??

Call signature:  relay._ffi_api.Any(*args, **kwargs)
Type:            PackedFunc
String form:     <tvm.runtime.packed_func.PackedFunc object at 0x7fb4e423a8e0>
File:            /media/pc/data/lxw/ai/tvm/xinetzone/tvm-book/__pypackages__/3.10/lib/tvm/runtime/packed_func.py
Source:         
class PackedFunc(PackedFuncBase):
    """The PackedFunc object used in TVM.

    Function plays an key role to bridge front and backend in TVM.
    Function provide a type-erased interface, you can call function with positional arguments.

    The compiled module returns Function.
    TVM backend also registers and exposes its API as Functions.

    The following are list of common usage scenario of tvm.runtime.PackedFunc.

    - Automatic exposure of C++ API into python
    - To call PackedFunc from python side
    - To call python callbacks to inspect results in generated code
    - Bring python hook into C++ backend

    See Also
    --------
    tvm.register_func: How to register global function.
